In [1]:
import string
from copy import deepcopy as copy

from IPython.display import Image, clear_output, display
from Libs.Beholder import *
from PIL import Image as pilImage
from tqdm.notebook import tqdm

In [2]:
a = Beholder(videoFrameGenerator=pullPhoneScreen)

a.addLayerModifer(
    Beholder_Layer_Chopper_Grayscale(name="gray", from_layer="image", enabled=True)
)

a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="gray_letters",
        from_layer="gray",
        x=304,
        y=1632,
        w=850,
        h=860,
        enabled=True,
    )
)

In [3]:
from pathlib import Path

import requests

w = Path("words.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://raw.githubusercontent.com/MariaLuo826/english_words/master/words_alpha.txt"
        ).content
    )
words = [l.upper() for l in w.read_text().splitlines()]
len(words)

370105

In [4]:
# Top words
w = Path("1ktopwords.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://gist.githubusercontent.com/deekayen/4148741/raw/98d35708fa344717d8eee15d11987de6c8e26d7d/1-1000.txt"
        ).content
    )
topwords = [l.upper() for l in w.read_text().splitlines()]
len(topwords)

1000

In [5]:
# wiki words
w = Path("wikiwords.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://gist.githubusercontent.com/h3xx/1976236/raw/bbabb412261386673eff521dddbe1dc815373b1d/wiki-100k.txt"
        ).content
    )
wikiwords = list(sorted(list(set([l.upper() for l in w.read_text().splitlines()]))))
len(wikiwords)

70926

In [6]:
# h3x words
w = Path("1ktopwords.lst")
if not w.exists():
    w.write_bytes(requests.get("").content)
topwords = [l.upper() for l in w.read_text().splitlines()]
len(topwords)

1000

In [7]:
a.matchers = {}
for l in string.ascii_uppercase:
    p = Path(f"./wordtrip_templates/{l}.png")
    if p.exists():
        print(f"Loading {l}")
        a.addMatcher(
            Beholder_Image_Matcher(
                name=l,
                layer="gray_letters",
                filename=str(p),
                threshhold=0.9,
                sensitivity=150,
            )
        )

for l in ["SH"]:
    p = Path(f"./wordtrip_templates/{l}.png")
    if p.exists():
        print(f"Loading {l}")
        a.addMatcher(
            Beholder_Image_Matcher(
                name=list(l),
                layer="gray_letters",
                filename=str(p),
                threshhold=0.9,
                sensitivity=150,
            )
        )

Loading A
Loading B
Loading C
Loading D
Loading E
Loading F
Loading G
Loading H
Loading I
Loading K
Loading L
Loading M
Loading N
Loading O
Loading P
Loading R
Loading S
Loading T
Loading U
Loading V
Loading W
Loading Y
Loading sh


In [8]:
def wordWorks(word, found_letters):
    found_letters_t = copy(found_letters)
    for letter in word:
        if letter in found_letters_t:
            found_letters_t.remove(letter)
        else:
            return False
    return True

In [9]:
def generateMoves(myword, matches):
    mymatches = copy(matches)
    moves = []
    for letter in myword:
        if letter in matches:
            if len(matches[letter]) > 0:
                moves.append(matches[letter][0].center)
                mymatches[letter].pop(0)

    return moves

In [10]:
min_len = 3
max_len = 5
eq_len = None

# Manual matcher setup 

In [11]:
matches = a.findMatches()
found_letters = []
for l in matches:
    for _ in range(0, len(matches[l])):
        found_letters.append(l)
print(found_letters)

matching_words = []
for word in tqdm(words):
    if len(word) < min_len or len(word) > max_len:
        continue
    if wordWorks(word, found_letters):
        matching_words.append(word)
matching_words = list(sorted(matching_words, key=len, reverse=True))
olen = 0
for word in matching_words:
    if len(word) != olen:
        print()
        olen = len(word)
    print(word, end=":")
    print(generateMoves(word, matches))

AddMatch D (363, 1858)
AddMatch E (824, 2201)
AddMatch I (474, 2172)
AddMatch N (922, 1839)
['D', 'E', 'I', 'N']


  0%|          | 0/370105 [00:00<?, ?it/s]


DINE:[(363, 1858), (474, 2172), (922, 1839), (824, 2201)]
ENID:[(824, 2201), (922, 1839), (474, 2172), (363, 1858)]
INDE:[(474, 2172), (922, 1839), (363, 1858), (824, 2201)]
NIDE:[(922, 1839), (474, 2172), (363, 1858), (824, 2201)]

DEI:[(363, 1858), (824, 2201), (474, 2172)]
DEN:[(363, 1858), (824, 2201), (922, 1839)]
DIE:[(363, 1858), (474, 2172), (824, 2201)]
DIN:[(363, 1858), (474, 2172), (922, 1839)]
END:[(824, 2201), (922, 1839), (363, 1858)]
IDE:[(474, 2172), (363, 1858), (824, 2201)]
IND:[(474, 2172), (922, 1839), (363, 1858)]
NED:[(922, 1839), (824, 2201), (363, 1858)]
NEI:[(922, 1839), (824, 2201), (474, 2172)]
NID:[(922, 1839), (474, 2172), (363, 1858)]


# Monkey Test

In [12]:
import socket
import subprocess
import sys


class Monkey:
    def __init__(self, create_monkey=True):
        self.create_monkey = create_monkey
        self.port = 9905
        self.p = None

    def __enter__(self):
        if self.p is not None:
            self.p.kill()
        adb(["forward", "--remove-all"])
        command = ["adb", "shell", "monkey", f"--port {self.port}"]
        self.p = shell(command, close_fds=True, returns_POpen=True)
        adb(f"forward tcp:{self.port} tcp:{self.port}")
        time.sleep(0.75)
        self.connect()
        return self

    def connect(self):
        HOST = "127.0.0.1"
        PORT = self.port
        self.s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.s.connect((HOST, PORT))

    def raw(self, s):
        try:
            self.s.sendall(f"{s}\r\n".encode("ascii"))
            # return self.s.recv(10)
        except Exception as e:
            print("raw", e)
            self.connect()
            return self.raw(s)

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if self.create_monkey:
            self.p.kill()
            self.p = None
            if self.s is not None:
                self.s.close()
                self.s = None
            print("Cleaning up")
            shell(["adb", "forward", "--remove-all"])

In [ ]:
with Monkey() as monkey:
    matches = a.findMatches()
    found_letters = []
    for l in matches:
        for _ in range(0, len(matches[l])):
            found_letters.append(l)
    print(found_letters)

    matching_words = []
    for word in tqdm(words):
        if len(word) < min_len or len(word) > max_len:
            continue
        if wordWorks(word, found_letters):
            matching_words.append(word)

    plen = 0
    matching_words = list(sorted(matching_words, key=len, reverse=True))
    for word in matching_words:
        print(word)
        x = y = 0
        path = generateMoves(word, matches)
        print(path)
        isDown = False
        for x, y in path:
            if not isDown:
                cmd = f"touch down {int(x)} {int(y)}"
                print(cmd)
                print(monkey.raw(cmd))
                isDown = True
                time.sleep(1)
                continue
            cmd = f"touch move {int(x)} {int(y)}"
            print(cmd)
            print(monkey.raw(cmd))
            time.sleep(0.2)
        cmd = f"touch up {int(x)} {int(y)}"
        print(cmd)
        print(monkey.raw(cmd))

        time.sleep(1)
        sleep(2)

  bash arg: --port
  bash arg: 9905


args: [--port, 9905]
 arg: "--port"
 arg: "9905"
arg="--port" mCurArgData="null" mNextArg=1 argwas="--port" nextarg="9905"
data="9905"


Error binding to network socket.
AddMatch D (363, 1858)
AddMatch E (824, 2201)
AddMatch I (474, 2172)
AddMatch N (922, 1839)
['D', 'E', 'I', 'N']


  0%|          | 0/370105 [00:00<?, ?it/s]

DINE
[(363, 1858), (474, 2172), (922, 1839), (824, 2201)]
touch down 363 1858
None
touch move 474 2172
raw [Errno 32] Broken pipe
None
touch move 922 1839
None
touch move 824 2201
None
touch up 824 2201
None
ENID
[(824, 2201), (922, 1839), (474, 2172), (363, 1858)]
touch down 824 2201
None
touch move 922 1839
raw [Errno 32] Broken pipe
None
touch move 474 2172
None
touch move 363 1858
None
touch up 363 1858
None
INDE
[(474, 2172), (922, 1839), (363, 1858), (824, 2201)]
touch down 474 2172
None
touch move 922 1839
raw [Errno 32] Broken pipe
None
touch move 363 1858
None
touch move 824 2201
None
touch up 824 2201
None
NIDE
[(922, 1839), (474, 2172), (363, 1858), (824, 2201)]
touch down 922 1839
None
touch move 474 2172
raw [Errno 32] Broken pipe
None
touch move 363 1858
None
touch move 824 2201
None
touch up 824 2201
None
DEI
[(363, 1858), (824, 2201), (474, 2172)]
touch down 363 1858
None
touch move 824 2201
raw [Errno 32] Broken pipe
None
touch move 474 2172
None
touch up 474 2172
None

# Research


In [ ]:
matches

In [ ]:
matches["T"][0].center

In [ ]:
matches["N"][0].center

shell input touchscreen swipe x1,y1, x2,y2

https://github.com/mattwilson1024/android-pattern-unlock/blob/master/unlock.sh

In [ ]:
matches = a.findMatches()
found_letters = []
for l in matches:
    for _ in range(0, len(matches[l])):
        found_letters.append(l)
print(found_letters)

matching_words = []
for word in tqdm(words):
    if len(word) < min_len or len(word) > max_len:
        continue
    if wordWorks(word, found_letters):
        matching_words.append(word)

matching_words = list(sorted(matching_words, key=len, reverse=True))
matching_words

In [ ]:
def StartTouch():
    print("d")
    adb("shell -c sendevent /dev/input/event3 3 57 14")


def SendCoordinates(x, y):
    print(x, y)
    adb(f"shell -c sendevent /dev/input/event3 3 53 {x}")
    adb(f"shell -c sendevent /dev/input/event3 3 54 {y}")
    adb("shell -c sendevent /dev/input/event3 3 58 57")
    adb("shell -c endevent /dev/input/event3 0 0 0")


def FinishTouch():
    print("u")
    adb("shell sendevent /dev/input/event3 3 57 4294967295")
    adb("shell sendevent /dev/input/event3 0 0 0")

In [ ]:
plen = 0
for word in matching_words:
    print(word)
    x = y = 0
    path = generateMoves(word, matches)
    print(path)
    isDown = False
    for x, y in path:
        if not isDown:
            StartTouch()
            time.sleep(1)
            isDown = True
        SendCoordinates(int(x), int(y))
        time.sleep(0.2)
    FinishTouch()
    sleep(2)